In [16]:
from sklearn.preprocessing import FunctionTransformer


def preprocess_data(X,y):
  
    X = X.drop(['CUST_ID', 'CREDIT_SCORE', 'T_EDUCATION_12', 'T_FINES_12', 'T_GAMBLING_12', 'T_HOUSING_12', 'T_TAX_12', 
                'T_TRAVEL_12', 'T_EDUCATION_6', 'T_ENTERTAINMENT_6', 'T_ENTERTAINMENT_12', 'T_GAMBLING_6', 
                'T_GROCERIES_6', 'T_HOUSING_6', 'T_EXPENDITURE_12', 'T_EXPENDITURE_6', 'R_GROCERIES_DEBT', 
                'INCOME', 'T_UTILITIES_6', 'T_FINES_6', 'R_FINES_INCOME', 'R_FINES_DEBT', 'CAT_GAMBLING', 
                'CAT_DEBT', 'CAT_MORTGAGE', 'CAT_SAVINGS_ACCOUNT', 'CAT_DEPENDENTS', 'R_FINES_SAVINGS', 
                'CAT_CREDIT_CARD'], axis=1, errors='ignore')
    
    
    if 'CAT_GAMBLING' in X.columns:
        X['CAT_GAMBLING'] = X['CAT_GAMBLING'].map({'No': 0, 'High': 2, 'Low': 1})
    
  
    clip_values = {
        'SAVINGS': 2500000,
        'DEBT': 4000000,
        'T_CLOTHING_12': 32000,
        'T_CLOTHING_6': 25000,
        'T_HEALTH_12': 25000,
        'T_HEALTH_6': 18000,
        'T_TRAVEL_12': 150000,
        'T_TRAVEL_6': 110000,
    }
    
    for column, max_value in clip_values.items():
        if column in X.columns:
            X[column] = X[column].clip(upper=max_value)
    
    return X,y


preprocessor = FunctionTransformer(preprocess_data)




In [17]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
import pandas as pd
data = pd.read_csv('../data/data.csv')

# Assuming `preprocess_data` is your preprocessing function
preprocessor = FunctionTransformer(preprocess_data)

# Setup the pipeline with the preprocessor and a placeholder for the classifier
pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('classifier', XGBClassifier(eval_metric='logloss', use_label_encoder=False))
])

# Define the parameter grid to search over
param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [3, 4, 5],
    # Add more parameters here as needed
}

# Setup the GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=1)

# Split your data
X = data.drop(['DEFAULT'], axis=1)
y = data['DEFAULT']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=420)

# Run the grid search
grid_search.fit(X_train, y_train)

# Best parameters and score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

# To make predictions
# y_pred = grid_search.predict(X_test)


Fitting 5 folds for each of 6 candidates, totalling 30 fits


ValueError: 
All the 30 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/glebbadejko/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/glebbadejko/anaconda3/lib/python3.11/site-packages/sklearn/pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/glebbadejko/anaconda3/lib/python3.11/site-packages/sklearn/pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_transform_one_cached(
                            ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/glebbadejko/anaconda3/lib/python3.11/s